# Exploratory analysis for NLP models

Vlad scraped glassdoor last year for job descriptions. Here is the raw data:

In [44]:
# packages needed
#!pip install langdetect
#!pip install nltk
#!pip install pyLDAvis
#!pip install gensim
#!pip install textblob
#!pip install googletrans
#!pip install prettyprinter
!

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from time import sleep
from langdetect import detect
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# import pyLDAvis
# import pyLDAvis.gensim  # don't skip this
from time import sleep
from pprint import pprint
import joblib

from 

%matplotlib inline

In [51]:
#df = pd.read_json("../../../raw_data/indeed/data_science_berlin_2021-03-02_1500.json")
df = joblib.load('../../../raw_data/indeed/indeed_proc.joblib')

In [52]:
df.head(5)

,job_title,job_text,company,location,job_info,query_text,tag_language,job_info_tokenized,job_text_tokenized,job_text_tokenized_titlecase,job_title_tokenized
0,Data Scientist / Matching Engineer (m/w/d),You are responsible for improvement of Taledo’...,Taledo,Berlin,Data Scientist / Matching Engineer (m/w/d)\nTa...,raw data/indeed,en,[datascientistmatchingengineermwdtaledobewertu...,[youareresponsibleforimprovementoftaledossearc...,[YouareresponsibleforimprovementofTaledosSearc...,[datascientistmatchingengineermwd]
1,(Junior) Data Engineer (f/m/x),Über die Stelle\nUnser Data Team braucht Unter...,Customlytics GmbH,Berlin,(Junior) Data Engineer (f/m/x)\nCustomlytics G...,raw data/indeed,de,[juniordataengineerfmxcustomlyticsgmbhberlinvo...,[überdiestelleunserdatateambrauchtunterstützun...,[ÜberdieStelleUnserDataTeambrauchtUnterstützun...,[juniordataengineerfmx]
2,Junior Data Scientist (m/w/d) docmetric,Kennziffer:\nreq6004\nStandort:\nBerlin\nJob S...,CompuGroup Medical,Berlin,Junior Data Scientist (m/w/d) docmetric\nCompu...,raw data/indeed,de,[juniordatascientistmwddocmetriccompugroupmedi...,[kennzifferreqstandortberlinjobsegmentsoftware...,[KennzifferreqStandortBerlinJobSegmentSoftware...,[juniordatascientistmwddocmetric]
3,Senior Software Engineer - Data Platform,We are looking for a Senior Software Engineer ...,Zalando SE,Berlin,Senior Software Engineer - Data Platform\nZala...,raw data/indeed,en,[seniorsoftwareengineerdataplatformzalandosebe...,[wearelookingforaseniorsoftwareengineerwithext...,[WearelookingforaSeniorSoftwareEngineerwithext...,[seniorsoftwareengineerdataplatform]
4,Werkstudent Software/Data Engineer (w/m/d) | J...,bakdata GmbH\nDie bakdata GmbH ist ein unabhän...,bakdata GmbH,Berlin,Werkstudent Software/Data Engineer (w/m/d) | J...,raw data/indeed,de,[werkstudentsoftwaredataengineerwmdjavapythona...,[bakdatagmbhdiebakdatagmbhisteinunabhängigerit...,[bakdataGmbHDiebakdataGmbHisteinunabhängigerIT...,[werkstudentsoftwaredataengineerwmdjavapythona...


In [50]:
df['job_text_tokenized']

0      [youareresponsibleforimprovementoftaledossearc...
1      [überdiestelleunserdatateambrauchtunterstützun...
2      [kennzifferreqstandortberlinjobsegmentsoftware...
3      [wearelookingforaseniorsoftwareengineerwithext...
4      [bakdatagmbhdiebakdatagmbhisteinunabhängigerit...
                             ...                        
801    [exocadisaleadingdentalsoftwarecompanycontinuo...
802    [startdatumwirsuchendichabsofortdiepositionist...
808    [thetechnicalsupportengineercanbebasedanywhere...
809    [wanttobeaherojoinoneoftheleadingglobalonlinef...
811    [theroleandyourimpactattiermaintainourwebshopp...
Name: job_text_tokenized, Length: 558, dtype: object

## 1. Preprocessing functions
done by Alex

In [5]:
df.shape

(324, 5)

In [6]:
## change case to lower
def to_lower(text):
    return text.lower()
# df["clean"] = df["job_text"].apply(to_lower)

In [7]:
## remove numbers from the corpus
def remove_number(text):
    text = ''.join(word for word in text if not word.isdigit())
    
    return text

# df["clean"]= df["clean"].apply(remove_number)

In [8]:
## remove special puncutation from text
def remove_punctuation(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    
    return text

# df["clean"] = df["clean"].apply(remove_punctuation)

In [9]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
## tag the language of the dataframe - IMP does 
from time import sleep
from langdetect import detect

def tag_language(text):
    ln = detect(text)
    
    return ln

# df["language"] = df["clean"].apply(tag_language)

In [11]:
# # text blob catches language with more accuracy I would use this one
# from textblob import TextBlob
# from time import sleep

# def language_id(text):
#     b = TextBlob(text)
#     blob = b.detect_language()
#     time.sleep(1)
    
#     return blob

# df["lang_blob"] = df["clean"].apply(language_id)

In [13]:
## lemmatize the output

def lemmatize_words(text):
    
    lemmatizer = WordNetLemmatizer()

    lemmatized = [lemmatizer.lemmatize(word) for word in text]

    return lemmatized

In [12]:
updated_stopwords = ['work', 'working', '’', 'berlin', 'team', 'teams', 'data', 
                       'skill', 'skills','new', 'diesen', 'zur', 'us', 'u', 'well', 
                       'bewerbung', 'job', 'weiter', 'help', 'melden', 'best']

## remove stopwords and tokenize the text
def remove_stopwords(text):

    stop_words = set(stopwords.words('english'))
    stop_words.update(updated_stopwords)
    #print(stop_words)
    
    word_tokens = word_tokenize(text) 
  
    text = [w for w in word_tokens if not w in stop_words] 
  
    return text


#df["clean"] = df["clean"].apply(remove_stopwords)

# import csv
# with open('stopwords.csv', 'w') as f:
#     for key in tmp.keys():
#         f.write("%s,%s\n"%(key,tmp[key]))

In [14]:
df = df.drop(0)
df.head(2)

,job_title,job_text,company,location,job_info
1,,Responsibilities\n\nAs working student (m/f/x)...,Aroundhome,- Berlin,Aroundhome6 Bewertungen - Berlin
2,,Aufgaben\nAls Werkstudent (m/w/d) IT arbeitest...,Aroundhome,- Berlin,"Aroundhome6 Bewertungen - Berlin\nTeilzeit, Pr..."


In [15]:
df.shape

(323, 5)

## 2. Exploration of LDA

Main aim: try out LDA combined with different tokenizing methods. Unsupervised method, requires no input from a human

Output: extract topics and keywords from job descriptions

### E_01 - simple LDA with gensim basic vectorizer
try approach from: https://medium.com/analytics-vidhya/data-science-job-search-using-nlp-and-lda-in-python-12ecbfac79f9

### Results:
best params:
- number of topics (28 according to coherence) - too many topics based on topic visulaization
- removed some stopwords 

next steps:
- LDA: try mallet wrapper, eta, alpha 
- try different vectorizers

In [16]:
df["clean"] = df["job_text"].apply(to_lower)
df["clean"]= df["clean"].apply(remove_number)
df["clean"] = df["clean"].apply(remove_punctuation)
df["language"] = df["clean"].apply(tag_language)
df["clean"] = df["clean"].apply(remove_stopwords)
df["clean"] = df["clean"].apply(lemmatize_words)

In [17]:
df.head()

,job_title,job_text,company,location,job_info,clean,language
1,,Responsibilities\n\nAs working student (m/f/x)...,Aroundhome,- Berlin,Aroundhome6 Bewertungen - Berlin,"[responsibility, student, mfx, close, collabor...",en
2,,Aufgaben\nAls Werkstudent (m/w/d) IT arbeitest...,Aroundhome,- Berlin,"Aroundhome6 Bewertungen - Berlin\nTeilzeit, Pr...","[aufgaben, al, werkstudent, mwd, arbeitest, du...",de
3,,Startdatum\n\nAb sofort\n\nDeine Aufgaben\n\nD...,Alexander Thamm GmbH,- Berlin,Alexander Thamm GmbH - Berlin\nFestanstellung,"[startdatum, ab, sofort, deine, aufgaben, dein...",de
4,Full Stack Developer (m/f/d),We’re Phiture: a leading mobile growth consult...,Phiture,- Berlin-Kreuzberg,Full Stack Developer (m/f/d)\nPhiture - Berlin...,"[phiture, leading, mobile, growth, consultancy...",en
5,,"We are 18,000+ employees strong, operating in ...",PRA Health Sciences,- Berlin,PRA Health Sciences - Berlin,"[employee, strong, operating, country, committ...",en


In [18]:
# seledct english descriptions and reset index
df_eng = df[df["language"] == "en"]
df_eng.reset_index(inplace=True)
df_eng = df_eng.rename(columns={'index': 'tmp_index'})
df_eng = df_eng.drop(columns='tmp_index')
df_eng.head(2)

,job_title,job_text,company,location,job_info,clean,language
0,,Responsibilities\n\nAs working student (m/f/x)...,Aroundhome,- Berlin,Aroundhome6 Bewertungen - Berlin,"[responsibility, student, mfx, close, collabor...",en
1,Full Stack Developer (m/f/d),We’re Phiture: a leading mobile growth consult...,Phiture,- Berlin-Kreuzberg,Full Stack Developer (m/f/d)\nPhiture - Berlin...,"[phiture, leading, mobile, growth, consultancy...",en


In [ ]:
df_eng.shape

In [21]:
def bag_of_words(descriptions):
    
    # create corpus
    descriptions = descriptions
    
    # make dictionary of words
    id2word = corpora.Dictionary(descriptions)
    
    # Term Document Frequency; description to bag fo words
    corpus = [id2word.doc2bow(text) for text in descriptions]
    
    return descriptions, id2word, corpus

descriptions, id2word, corpus = bag_of_words(df_eng['clean'])

In [30]:
pprint(id2word.token2id)

{'aanonsen': 4740,
 'ab': 2441,
 'abandoned': 5770,
 'aber': 2442,
 'ability': 304,
 'able': 1102,
 'abovementioned': 3569,
 'abreast': 5246,
 'absolute': 0,
 'abstandsvorschriften': 7128,
 'abstract': 3275,
 'abstraction': 994,
 'abstractly': 1689,
 'abu': 6358,
 'academia': 2100,
 'academic': 1103,
 'academy': 1184,
 'acams': 4002,
 'acc': 3570,
 'accelerate': 2584,
 'accelerated': 4870,
 'acceleration': 6422,
 'accelerator': 853,
 'accept': 3891,
 'acceptance': 3636,
 'accepted': 5351,
 'accepting': 4311,
 'access': 551,
 'accessed': 4441,
 'accessibility': 5522,
 'accessible': 995,
 'accessory': 5451,
 'accident': 1436,
 'accommodate': 2175,
 'accommodation': 3637,
 'accompanied': 1886,
 'accompanying': 6855,
 'accomplish': 2368,
 'accomplishment': 6510,
 'accordance': 1740,
 'according': 1185,
 'accordingly': 3544,
 'account': 130,
 'accountability': 4514,
 'accountable': 1104,
 'accountancy': 427,
 'accountant': 1610,
 'accounting': 428,
 'accredited': 1611,
 'accuracy': 1690,
 '

In [ ]:
# # View endoded words
# print(corpus[0][:10]) 

# # access words
# id2word[0]

In [ ]:
# tmp = pd.Series(descriptions[0]).sort_values()
# tmp[:50]

In [ ]:

# # find words per topic 
# for cp in range(3):#len(corpus)):
#     word_freq = corpus[cp]
#     for id, freq in word_freq:
#         print((id2word[id], freq))

In [ ]:
#df_eng['job_text'][0]

In [ ]:
#df_eng['job_text'][2]

In [31]:
# crossvalidate with coherence scores
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics, random_state=42,
                          #update_every=1, #passes=10,
                          alpha='auto')
        #model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# run models with different topics
limit=40; start=2; step=2;
model_list, coherence_values = compute_coherence_values(dictionary=id2word, 
                                                        corpus=corpus, 
                                                        texts=df_eng['clean'], 
                                                        start=start, 
                                                        limit=limit, 
                                                        step=step)

In [ ]:
coherence_values

In [ ]:
x = list(range(start, limit, step))
plt.plot(x, coherence_values)
plt.title('Coherence values')
plt.xlabel('Num Topics')
plt.ylabel('Coherence score')
# type(x) 
#len(coherence_values)

In [ ]:
# implement LDA

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=28, 
                                           random_state=42,
                                           #update_every=1,
                                           #passes=10,
                                           alpha='auto')
                                           #per_word_topics=True)

In [ ]:
# view topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=df_eng['clean'], dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# # visualise w/ pyLDAvis
# import pyLDAvis
# import pyLDAvis.gensim  # don't skip this

# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
# vis

### E_02 - simple LDA with Mallet wrapper
approach from: https://medium.com/analytics-vidhya/data-science-job-search-using-nlp-and-lda-in-python-12ecbfac79f9

### Results (CHANGE HERE):
best params:
- number of topics (28 according to coherence) - too many topics based on topic visulaization
- removed some stopwords 

next steps:
- LDA: try mallet wrapper, eta, alpha 
- try different vectorizers

In [37]:
from gensim.models.wrappers import LdaMallet

In [32]:
descriptions, id2word, corpus = bag_of_words(df_eng['clean'])

In [33]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=28, 
                                           random_state=42,
                                           #update_every=1,
                                           #passes=10,
                                           alpha='auto')
                                           #per_word_topics=True)

In [42]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = '../../../packages_tmp/mallet-2.0.8/bin/mallet' # update this path

#path_to_mallet_binary = "/path/to/mallet/binary"
#mallet_path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=id2word)

CalledProcessError: Command '../../../packages_tmp/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /var/folders/gz/pnb32qk93rvbj_cd8w1ffpdr000nmp/T/8f1855_corpus.txt --output /var/folders/gz/pnb32qk93rvbj_cd8w1ffpdr000nmp/T/8f1855_corpus.mallet' returned non-zero exit status 1.

In [ ]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

In [ ]:
# view topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=df_eng['clean'], dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# visualise w/ pyLDAvis
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis